# Visualization of MS spectrum

## Imports

In [1]:
%matplotlib inline

from pathlib import Path

from matplotlib import pyplot as plt,__version__ as plt_version
from pkg_resources import get_distribution  # Comes with setuptools.
from pyteomics import mzml, auxiliary, mass
from pyopenms import __version__ as pyopenms_version, IdXMLFile


import ipywidgets as widgets
from IPython.display import clear_output

# Libraries versions.
print(f"Matplotlib version : {plt_version}")
try:
    print(f"Pyteomics version: {get_distribution('pyteomics').version}")
except:
    print("Pyteomics version not found")
print(f"PyOpenMS version: {pyopenms_version}")

Matplotlib version : 3.1.1
Pyteomics version: 4.1.2
PyOpenMS version: 2.4.0


## Parsing the mzML 

Choosing the file is done programmatically as the file upload may not work due to the size
of the mzML files.

In [2]:
# Get the directory with the mass spectrometry files.
DATA_DIR = Path("..").resolve().joinpath("data/")

# mzML file.
MZML_FILE = DATA_DIR.joinpath("Q1_5_truncated.mzML")

In [3]:
ms1_scans = []
ms1_mz_arrays = []
ms1_intensity_arrays = []
ms1_retention_times = []

ms2_scans = []
ms2_mz_arrays = []
ms2_intensity_arrays = []
ms2_retention_times = []

In [4]:
# Get the MS1 and MS2 spectra.
with mzml.read(str(MZML_FILE)) as reader:
    for spectrum in reader:
        if spectrum["ms level"] == 1:
            ms1_scans.append(spectrum["id"])
            ms1_mz_arrays.append(spectrum["m/z array"])
            ms1_intensity_arrays.append(spectrum["intensity array"])
            ms1_retention_times.append(spectrum["scanList"]["scan"][0]["scan start time"])
        elif spectrum["ms level"] == 2:
            ms2_scans.append(spectrum["id"])
            ms2_mz_arrays.append(spectrum["m/z array"])
            ms2_intensity_arrays.append(spectrum["intensity array"])
            ms2_retention_times.append(spectrum["scanList"]["scan"][0]["scan start time"])

print(f"File parsed. There are {len(ms1_scans)} MS1 spectrums and {len(ms2_scans)} MS2 spectrums.")

File parsed. There are 1421 MS1 spectrums and 79 MS2 spectrums.


## Widget management

In [5]:
# Interactive display of MS1 spectrum.
file_box = widgets.Select(
    options=ms1_retention_times,
    # Writing Retention time is too long for the default label.
    description='RT (minutes):',
    disabled=False,
)

button = widgets.Button(description='Display MS1',
                        button_style="info")


# Configuring the widgets interactions.

out = widgets.Output()

# Display the image when the button is clicked.  Is rather slow.
def display_image(_):
    with out:
        clear_output()
        # Supposes the retention time have same index as arrays.
        spectrum_index = ms1_retention_times.index(file_box.value)
        plt.figure()
        # The width needs to be adjusted for some arrays with extreme peaks.
        plt.bar(ms1_mz_arrays[spectrum_index], ms1_intensity_arrays[spectrum_index])
        plt.xlabel("m/z")
        plt.ylabel("intensity")
        plt.title(f"MS1 spectrum")
        plt.show()

button.on_click(display_image)

## Main widget

In [6]:
# Main widget.
box = widgets.VBox([file_box, button, out])
box

## Using pyOpenMS to parse the idXML

In [7]:
# Get IdXML file in data/.
IDXML_FILE = DATA_DIR.joinpath("1937004_Q1_5.idXML")

# Threshold of relative abundance for idXML hits.
ISOTOPE_THRESHOLD = 5e-3

In [8]:
# Load the file and get its content.
protein_ids = []
peptide_ids = []

IdXMLFile().load(str(IDXML_FILE), protein_ids, peptide_ids)

In [9]:
# Retrieve the identifiers connecting to the mzML.
# Decode to go from binary to utf8.
idxml_scans = [scan.getMetaValue("spectrum_reference").decode() for scan in peptide_ids]
print(f"There are {len(idxml_scans)} identifiers in the idXML")

id_intersection = set(idxml_scans).intersection(set(ms2_scans))
print(f"There are {len(id_intersection)} intersecting MS2 IDs")

There are 31920 identifiers in the idXML
There are 9 intersecting MS2 IDs


In [10]:
# RT of MS2 with peptides found
ms2_shared_retention_times = [scan.getRT() for scan in peptide_ids
 if scan.getMetaValue("spectrum_reference").decode() in id_intersection]
print(f"RT of MS2 with peptides in the idXML: \n{ms2_shared_retention_times}")

RT of MS2 with peptides in the idXML: 
[362.73063, 362.804376, 363.02676, 363.1005, 363.326496, 363.401244, 363.475122, 364.050372, 379.877004]


## Comparison between MS2 and peptide in IdXML

In [11]:
# Interactive display of MS2 spectrum with the idXML comparison.
# Only take MS2 for which peptides are found.
ms2_RT_chooser = widgets.Select(
    options=ms2_shared_retention_times,
    description='RT (minutes):',
    disabled=False,
)

comparison_button = widgets.Button(description='Display',
                                   button_style="info")


# Configuring the widgets interactions.
ms2_out = widgets.Output()

# Display the image when the button is clicked.  Is rather slow.
def display_comparison(_):
    xlabel = "m/z"
    with ms2_out:
        clear_output()
        # Supposes the retention time have same index as arrays.
        spectrum_index = ms2_retention_times.index(ms2_RT_chooser.value)

        # Find hits in the IdXML corresponding to the scan.
        shared_scan = ms2_scans[spectrum_index]
        scan_found = False
        # Iterate over IdXML scans.
        for idxml_index in range(len(idxml_scans)):
            if idxml_scans[idxml_index] == shared_scan:
                scan_found = True
                # Take the first hit sequence found as a string.
                hits = peptide_ids[idxml_index].getHits()
                hit_seq = hits[0].getSequence().toString().decode()
                hit_charge = hits[0].getCharge()
                #print(f"Found sequence {hit_seq}")

        # MS2 spectrum.
        ms2 = plt.subplot(1,2,1)
        plt.bar(ms2_mz_arrays[spectrum_index], ms2_intensity_arrays[spectrum_index])
        plt.xlabel("m/z")
        plt.ylabel("intensity")

        # IdXML result.
        if not scan_found:
            hit_seq = ""
            xlabel = "Peptide not found"

        hit = plt.subplot(1,2,2)
        isotopes = [isotope for isotope in mass.isotopologues(hit_seq,
                                                              isotope_threshold=ISOTOPE_THRESHOLD)]

        relative_abundances = [mass.isotopic_composition_abundance(isotope) for isotope in isotopes]
        plt.bar([isotope.mass()/hit_charge for isotope in isotopes], relative_abundances)
        plt.xlabel(xlabel)
        plt.ylabel("Relative abundance")
        hit.yaxis.set_label_position("right")
        
        plt.title(f"Comparison between experimental and '{hit_seq}' spectrum")
        plt.show()

comparison_button.on_click(display_comparison)

In [12]:
# Main widget.
comparison_box = widgets.VBox([ms2_RT_chooser, comparison_button, ms2_out])
comparison_box

## Storing a smaller mzML

In [13]:
from pyopenms import MzMLFile, MSExperiment

In [14]:
"""
# Get the mzml file.
DATA_DIR = Path("..").resolve().joinpath("data/")

mzml_file = DATA_DIR.joinpath("1937004_Q1_5.mzML")

# Get IdXML file
idxml_file = DATA_DIR.joinpath("1937004_Q1_5.idXML")

# Storing the MzML content
exp = MSExperiment()
MzMLFile().load(str(mzml_file), exp)

spectrums = []
max_spectrum = 1500

e = MSExperiment()

for i, spec in enumerate(exp):
    if i >= max_spectrum:
        break
    else:
        e.addSpectrum(spec)

MzMLFile().store("smaller.mzML", e)
"""

'\n# Get the mzml file.\nDATA_DIR = Path("..").resolve().joinpath("data/")\n\nmzml_file = DATA_DIR.joinpath("1937004_Q1_5.mzML")\n\n# Get IdXML file\nidxml_file = DATA_DIR.joinpath("1937004_Q1_5.idXML")\n\n# Storing the MzML content\nexp = MSExperiment()\nMzMLFile().load(str(mzml_file), exp)\n\nspectrums = []\nmax_spectrum = 1500\n\ne = MSExperiment()\n\nfor i, spec in enumerate(exp):\n    if i >= max_spectrum:\n        break\n    else:\n        e.addSpectrum(spec)\n\nMzMLFile().store("smaller.mzML", e)\n'

## Append data to a HDF5

In [15]:
import pandas as pd  # The pytables dependency is needed
# from collections import defaultdict
import numpy as np

In [ ]:
# Convert values to HDF5.  Beware if HDF5 already exists
MS1_values = []
store = pd.HDFStore('store.h5')
chunk_size = 2000

# Supposes intensities and mz have same order.  Iterate over RT.
for i in range(len(ms1_retention_times)):
    # Dict of mz:intensity for MS1 at each recorded RT.  The mz is rounded to avoid too many mz.
    MS1_values.append({round(mz):intensity for mz, intensity in zip(ms1_mz_arrays[i],
                                                                    ms1_intensity_arrays[i])})

    # Dump information in the hdf5 every 2000 elements.
    # HDF5 also has a size limit for the header (column number).
    if i % chunk_size == 0:
        df = pd.DataFrame(MS1_values, index=ms1_retention_times[:i], columns=range(300, 1700, 1))
        # Replace Nan by 0.
        df.fillna(0, inplace=True)
        store.append("df", df)
        MS1_values = []
        # Temporary break.
        if i >= chunk_size:
            break
            
print("HDF5 finished")

In [ ]:
store.get('df')

In [ ]:
import vaex
print(vaex.__version__)

In [ ]:
df = vaex.open("store.h5")